In [1]:
import os
from pathlib import Path
import numpy as np
from Bio import SeqIO, Align
from Bio.Seq import Seq

from local.constants import WORKSPACE_ROOT
from local.caching import load, save
CACHE = Path("./cache")

In [2]:
contigs = []

def get_key(e):
    _q = e.qualifiers
    key = None
    for k in ["gene", "product"]:
        if k in _q:
            key = _q[k][0]
            break
    assert key is not None
    return key

for g in SeqIO.parse(f"{WORKSPACE_ROOT}/data/ecoli/reference_genomes/EPI300/epi300.pgap.gb", "genbank"):
    for e in g.features:
        if e.type != "CDS": continue
        k = get_key(e)
        # if k != "araC": continue
        if k != "trfA": continue
        seq = e.extract(g.seq)
        loc = e.location
        print(k, seq)
        break

trfA ATGAATCGGACGTTTGACCGGAAGGCATACAGGCAAGAACTGATCGACGCGGGGTTTTCCGCCGAGGATGCCGAAACCATCGCAAGCCGCACCGTCATGCGTGCGCCCCGCGAAACCTTCCAGTCCGTCGGCTCGATGGTCCAGCAAGCTACGGCCAAGATCGAGCGCGACAGCGTGCAACTGGCTCCCCCTGCCCTGCCCGCGCCATCGGCCGCCGTGGAGCGTTCGCGTCGTCTCGAGCAGGAGGCGGCAGGTTTGGCGAAGTCGATGACCATCGACACGCGAGGAACTATGACGACCAAGAAGCGAAAAACCGCCGGCGAGGGCCTGGCAAAACAGGTCAGCGAGGCCAAGCAGGCCGCGTTGCTGAAACACACGAAGCAGCAGATCAAGGAAATGCAGCTTTCCTTGTTCGATATTGCGCCGTGGCCGGACACGATGCGAGCGATGCCAAACGACACGGCCCGCTCTGCCCTGTTCACCACGCGCAACAAGAAAATCCCGCGCGAGGCGCTGCAAAACAAGGTCATTTTCCACGTCAACAAGGACGTGAAGATCACCTACACCGGCGTCGAGCTGCGGGCCGACGATGACGAACTGGTGTGGCAGCAGGTGTTGGAGTACGCGAAGCGCACCCCTATCGGCGAGCCGATCACCTTCACGTTCTACGAGCTTTGCCAGGACCTGGGCTGGTCGATCAATGGCCGGTATTACACGAAGGCCGAGGAATGCCTGTCGCGCCTACAGGCGACGGCGATGGACTTCACGTCCGACCGCGTTGGGCACCTGGAATCGGTGTCGCTGCTGCACCGCTTCCGCGTCCTGGACCGTGGCAAGAAAACGTCCCGTTGCCAGGTCCTGATCGACGAGGAAATCGTCGTGCTGTTTGCTGGCGACCACTACACGAAATTCATATGGGAGAAGTACCGCAAGCTGTCGCCGACGGCCCGACGGATGTTCGACTATTTCAGCTCGCACCGGGAGCCGTACCCGCT

In [17]:
trfa = """
>sp|P07676|TRFA_ECOLX Plasmid replication initiator protein TrfA OS=Escherichia coli OX=562 GN=trfA PE=1 SV=1
MNRTFDRKAYRQELIDAGFSAEDAETIASRTVMRAPRETFQSVGSMVQQATAKIERDSVQ
LAPPALPAPSAAVERSRRLEQEAAGLAKSMTIDTRGTMTTKKRKTAGEDLAKQVSEAKQA
ALLKHTKQQIKEMQLSLFDIAPWPDTMRAMPNDTARSALFTTRNKKIPREALQNKVIFHV
NKDVKITYTGVELRADDDELVWQQVLEYAKRTPIGEPITFTFYELCQDLGWSINGRYYTK
AEECLSRLQATAMGFTSDRVGHLESVSLLHRFRVLDRGKKTSRCQVLIDEEIVVLFAGDH
YTKFIWEKYRKLSPTARRMFDYFSSHREPYPLKLETFRLMCGSDSTRVKKWREQVGEACE
ELRGSGLVEHAWVNDDLVHCKR
"""
trfa = Seq(''.join([l for l in trfa.splitlines() if l and l[0] != ">"]))
trfa

Seq('MNRTFDRKAYRQELIDAGFSAEDAETIASRTVMRAPRETFQSVGSMVQQATAKI...CKR')

In [ ]:
epi300 = np.array([c for c in seq.translate()])
ref = np.array([c for c in trfa])

In [21]:
print(seq.translate())
print(trfa)
# print("-"*59)
print("-"*203)

MNRTFDRKAYRQELIDAGFSAEDAETIASRTVMRAPRETFQSVGSMVQQATAKIERDSVQLAPPALPAPSAAVERSRRLEQEAAGLAKSMTIDTRGTMTTKKRKTAGEGLAKQVSEAKQAALLKHTKQQIKEMQLSLFDIAPWPDTMRAMPNDTARSALFTTRNKKIPREALQNKVIFHVNKDVKITYTGVELRADDDELVWQQVLEYAKRTPIGEPITFTFYELCQDLGWSINGRYYTKAEECLSRLQATAMDFTSDRVGHLESVSLLHRFRVLDRGKKTSRCQVLIDEEIVVLFAGDHYTKFIWEKYRKLSPTARRMFDYFSSHREPYPLKLETFRLMCGSDSTRVKKWREQVGEACEELRGSGLVEHAWVNDDLVHCKR*
MNRTFDRKAYRQELIDAGFSAEDAETIASRTVMRAPRETFQSVGSMVQQATAKIERDSVQLAPPALPAPSAAVERSRRLEQEAAGLAKSMTIDTRGTMTTKKRKTAGEDLAKQVSEAKQAALLKHTKQQIKEMQLSLFDIAPWPDTMRAMPNDTARSALFTTRNKKIPREALQNKVIFHVNKDVKITYTGVELRADDDELVWQQVLEYAKRTPIGEPITFTFYELCQDLGWSINGRYYTKAEECLSRLQATAMGFTSDRVGHLESVSLLHRFRVLDRGKKTSRCQVLIDEEIVVLFAGDHYTKFIWEKYRKLSPTARRMFDYFSSHREPYPLKLETFRLMCGSDSTRVKKWREQVGEACEELRGSGLVEHAWVNDDLVHCKR
-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------


In [ ]:
# PPD-Prokaryotic Promoter Database
# http://lin-group.cn/database/ppd/browse.php?table=promoters&id=48104
araBp = "ccataagattagcggatcctacctgacgctttttatcgcaactctctactgtttctccatAcccgtttttttggatggagt".upper()

81

In [ ]:
ws = Path("./cache/araBp")
# with open(ws/"s.fa", "w") as f:
#     f.write(">epi300\n")
#     f.write(str(g.seq))

In [203]:
os.system(f"""\
    cd {ws}
    # makeblastdb -in s.fa \
    #     -dbtype nucl \
    #     -out ./db/s
    # echo ">araBp" >q.fa
    # echo "{araBp}">>q.fa
    blastn -query ./q.fa -db ./db/s -out ./araBp.out -outfmt 6
    cat araBp.out
""")
with open(ws/"./araBp.out") as f:
    toks = f.readline().split("\t")
    ara_start = int(toks[8])-1
    # print(toks[8], toks[9])

araBp	epi300	98.765	81	1	0	1	81	3390306	3390386	5.84e-36	145


In [192]:
loc.start,loc.end

(ExactPosition(3390442), ExactPosition(3391591))

In [230]:
test = g.seq[loc.start:loc.end]
print(test == seq)
start, end = ara_start-loc.start, 6
print(start, end)
# start, end = -len(araBp)+3, 3
upstream = g.seq[loc.start+start:loc.start+end]
# upstream = g.seq[loc.end-end:3390386].reverse_complement()
print(upstream)
markers = np.zeros(end-start, dtype="U1")
markers.fill(" ")
def mark(i, s):
    i -= start
    markers[i:i+len(s)] = [c for c in s]
mark(0, "|START")
plus1 = -61
# mark(start, "-35, -10, +1 are guesses >>>")
mark(plus1, "|+1")
mark(plus1-10, "|-10")
mark(plus1-35, "|-35")
mark(-53, "|RBS")
mark(plus1-47, "*")
# mark(start, f"|{start}")
# p = start+len(araBp)-1
# mark(p, f"|{p}")
print(Seq(araBp))
print("".join(markers))

True
-137 6
CCATAAGATTAGCGGATCCTACCTGACGCCTTTTATCGCAACTCTCTACTGTTTCTCCATACCCGTTTTTTTGGATGGAGTGAAAGGAGGAATTCACCATGGCTCTAAATTCCGGGGCAATCCCGCAAGGAGGGTGAATGAAT
CCATAAGATTAGCGGATCCTACCTGACGCTTTTTATCGCAACTCTCTACTGTTTCTCCATACCCGTTTTTTTGGATGGAGT
                             *           |-35                     |-10      |+1     |RBS                                                 |START


In [ ]:
# test = g.seq[loc.start:loc.end].reverse_complement()
# print(test == seq)
# start, end = loc.end-3390386, 6
# # start, end = -len(araBp)+3, 3
# upstream = g.seq[loc.end-end:loc.end-start].reverse_complement()
# # upstream = g.seq[loc.end-end:3390386].reverse_complement()
# print(upstream)
# markers = np.zeros(end-start, dtype="U1")
# markers.fill(" ")
# def mark(i, s):
#     i -= start
#     markers[i:i+len(s)] = [c for c in s]
# mark(0, "|+1")
# mark(-10, "|-10")
# mark(-35, "|-35")
# mark(start, f"|{start}")
# p = start+len(araBp)-1
# mark(p, f"|{p}")
# print(Seq(araBp).reverse_complement())
# print("".join(markers))

False



ValueError: negative dimensions are not allowed